# Regression

In [20]:
import pandas as pd
import numpy as np
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
genomes = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="genomics_cannabis")

In [21]:
bq_assistant = BigQueryHelper("bigquery-public-data", "genomics_cannabis")
bq_assistant.list_tables()

['MNPR01_201703',
 'MNPR01_reference_201703',
 'MNPR01_transcriptome_201703',
 'cs10_gff',
 'cs3k_project_info',
 'cs3k_vcf_cs10_dv090',
 'sample_info_201703']

In [22]:
query = """SELECT
  variant_id, alternate_bases, quality, type, 
  AB, ABP, AC, AF, AN, AO, DP, DPB, DPRA, EPP, 
  EPPR, GTI, MQM, MQMR, NS, NUMALT, ODDS, PAO,
  PQA, PQR, PRO, QA, QR, RO, RPL, RPP, RPPR, 
  RPR, RUN, SAF, SAP, SAR, SRF, SRP, SRR
FROM
  `bigquery-public-data.genomics_cannabis.MNPR01_201703` v
limit 10000000;"""
response = genomes.query_to_pandas_safe(query, max_gb_scanned=100)
response.head(10)

C:\Users\sharm\AppData\Roaming\Python\Python311\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,variant_id,alternate_bases,quality,type,AB,ABP,AC,AF,AN,AO,...,RPP,RPPR,RPR,RUN,SAF,SAP,SAR,SRF,SRP,SRR
0,CKXG8eKP9qOf8wESIGdpfDEwOTg0ODk3NTJ8Z2J8TU5QUj...,[CGGG],50.30730,[ins],[0.0],[0.0],[2],[1.0],2,[2],...,[7.35324],0.0,[0],[1],[2],[7.35324],[0],0,0.0,0
1,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTA0ODB8Z2J8TU5QUj...,[T],1.31260,[snp],[0.0],[0.0],[2],[1.0],2,[5],...,[6.91895],0.0,[4],[1],[5],[13.8677],[0],0,0.0,0
2,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTAxMTF8Z2J8TU5QUj...,[A],104.88600,[snp],[0.0],[0.0],[2],[1.0],2,[5],...,[13.8677],0.0,[5],[1],[0],[13.8677],[5],0,0.0,0
3,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTY0MDF8Z2J8TU5QUj...,[AATC],9.99844,[complex],[0.0],[0.0],[2],[1.0],2,[5],...,[13.8677],0.0,[0],[1],[0],[13.8677],[5],0,0.0,0
4,CKXG8eKP9qOf8wESIGdpfDEwOTg0ODg3NDJ8Z2J8TU5QUj...,[A],84.39300,[snp],[0.0],[0.0],[2],[1.0],2,[5],...,[13.8677],0.0,[0],[1],[0],[13.8677],[5],0,0.0,0
5,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTcwOTh8Z2J8TU5QUj...,[T],128.04000,[snp],[0.0],[0.0],[2],[1.0],2,[5],...,[3.44459],0.0,[2],[1],[2],[3.44459],[3],0,0.0,0
6,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTU0MDF8Z2J8TU5QUj...,[A],60.09690,[snp],[0.0],[0.0],[2],[1.0],2,[2],...,[7.35324],0.0,[0],[1],[1],[3.0103],[1],0,0.0,0
7,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTUxMTJ8Z2J8TU5QUj...,[A],41.97960,[snp],[0.0],[0.0],[2],[1.0],2,[5],...,[3.44459],0.0,[3],[1],[2],[3.44459],[3],0,0.0,0
8,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTQwMjF8Z2J8TU5QUj...,[T],33.54030,[snp],[0.0],[0.0],[2],[1.0],2,[6],...,[16.0391],0.0,[6],[1],[6],[16.0391],[0],0,0.0,0
9,CKXG8eKP9qOf8wESIGdpfDEwOTg0OTYxNjR8Z2J8TU5QUj...,[G],28.27190,[snp],[0.0],[0.0],[2],[1.0],2,[2],...,[7.35324],0.0,[0],[1],[0],[7.35324],[2],0,0.0,0


I shall run regression to determine my DNA samples based on proteins. 

In [23]:
response["DNA_A"] = response["alternate_bases"].str[0].str.count('A')

In [24]:
response["DNA_C"] = response["alternate_bases"].str[0].str.count('C')

In [25]:
response["DNA_G"] = response["alternate_bases"].str[0].str.count('G')

In [26]:
response["DNA_T"] = response["alternate_bases"].str[0].str.count('T')

Now I shall adjust the proteins for easy analysis. 

In [27]:
proteins = ["AB", "ABP", "AC", "AF", "AO", "DPRA", "EPP", "MQM", "PAO",
            "PQA", "QA", "RPL", "RPP", 
            "RPR", "RUN", "SAF", "SAP", "SAR", 
]

for protein in proteins:
    print(protein)
    response[protein] = response[protein].str[0]

AB
ABP
AC
AF
AO
DPRA
EPP
MQM
PAO
PQA
QA
RPL
RPP
RPR
RUN
SAF
SAP
SAR


In [28]:
response["type"] = response["type"].str[0]

In [29]:
response["type"].value_counts()

type
snp        7269191
complex    1504631
mnp         558953
ins         384535
del         282690
Name: count, dtype: int64

```type``` is a categorical variable, so I shall need to dummify it. 

In [30]:
dummies = pd.get_dummies(response["type"], dtype="int")
dummies

,complex,del,ins,mnp,snp
0,0,0,1,0,0
1,0,0,0,0,1
2,0,0,0,0,1
3,1,0,0,0,0
4,0,0,0,0,1
...,...,...,...,...,...
9999995,1,0,0,0,0
9999996,0,0,0,0,1
9999997,0,0,0,0,1
9999998,0,0,0,0,1


Now I shall define what I am regressing upon.

In [31]:
y = response[["DNA_A", "DNA_C", "DNA_G", "DNA_T"]]

In [32]:
X = pd.concat([
    response[["AB", "ABP", "AC", "AF", "AN", "AO", "DP", "DPB", "DPRA", "EPP", 
              "EPPR", "GTI", "MQM", "MQMR", "NS", "NUMALT", "ODDS", "PAO",
              "PQA", "PQR", "PRO", "QA", "QR", "RO", "RPL", "RPP", "RPPR", 
              "RPR", "RUN", "SAF", "SAP", "SAR", "SRF", "SRP", "SRR"
    ]], 
    dummies
], axis=1)

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=1337)
LR = LinearRegression()
LR.fit(X_train, y_train)
train_score = LR.score(X_train, y_train)
test_score = LR.score(X_test, y_test)
print(f'Train score {train_score}, test score {test_score}')

Train score 0.17884205941623288, test score 0.17699595227058326


I've achieved a test score of 16.8%.

In [35]:
X.columns

Index(['AB', 'ABP', 'AC', 'AF', 'AN', 'AO', 'DP', 'DPB', 'DPRA', 'EPP', 'EPPR',
       'GTI', 'MQM', 'MQMR', 'NS', 'NUMALT', 'ODDS', 'PAO', 'PQA', 'PQR',
       'PRO', 'QA', 'QR', 'RO', 'RPL', 'RPP', 'RPPR', 'RPR', 'RUN', 'SAF',
       'SAP', 'SAR', 'SRF', 'SRP', 'SRR', 'complex', 'del', 'ins', 'mnp',
       'snp'],
      dtype='object')

In [36]:
LR.coef_

array([[ 1.27839475e-01, -4.21391799e-04,  8.80481950e+01,
        -1.76010867e+02, -2.64376047e+02,  8.21121599e+05,
         1.69477836e-03, -2.50617677e-04,  4.97658945e+01,
        -1.95552669e-04,  1.27042240e-04,  4.12802073e-01,
        -9.92560701e-05, -1.57602296e-03,  8.18419622e-01,
         5.02460476e-02, -3.14842595e-04,  2.82565214e-02,
        -7.74123110e-04,  7.11455457e-04, -2.80579004e-02,
        -9.64767259e-05,  1.72896293e-04, -7.50448690e+02,
        -4.09314933e+05, -9.35543561e-05, -7.64419261e-05,
        -4.09314934e+05, -2.62766633e-02, -4.11806664e+05,
         6.34156841e-05, -4.11806662e+05,  7.50444212e+02,
         4.30582020e-04,  7.50439570e+02,  6.52883970e-01,
         3.92240680e-01,  1.17078336e+00, -4.33182255e-01,
        -9.27874405e-01],
       [-2.16269300e-01, -3.46238095e-04,  1.44640354e+02,
        -2.89311848e+02, -4.34507514e+02,  1.34953037e+06,
         3.63094750e-04,  1.86196111e-04,  8.17912792e+01,
        -2.04166794e-04, -9.05

In [37]:
coeffs = pd.DataFrame(data=LR.coef_, columns=X.columns, index=y.columns)

In [38]:
coeffs.transpose()

,DNA_A,DNA_C,DNA_G,DNA_T
AB,0.127839,-2.162693e-01,-2.185484e-01,1.294564e-01
ABP,-0.000421,-3.462381e-04,-2.724761e-04,-2.383933e-04
AC,88.048195,1.446404e+02,-4.568925e+02,4.003236e+02
AF,-176.010867,-2.893118e+02,9.137535e+02,-8.005721e+02
AN,-264.376047,-4.345075e+02,1.372374e+03,-1.202400e+03
AO,821121.598934,1.349530e+06,-4.262436e+06,3.734517e+06
DP,0.001695,3.630947e-04,3.265299e-04,1.389953e-03
DPB,-0.000251,1.861961e-04,2.171785e-04,-4.513558e-04
DPRA,49.765894,8.179128e+01,-2.583344e+02,2.263387e+02
EPP,-0.000196,-2.041668e-04,-2.001539e-04,-1.793918e-04


The strongest coefficients are the proteins AO, RPL, RPR, SAF, and SAR. 

Notably, there is also a strong correlation between all four of the DNA types in terms of protein coefficients between them. 